In [1]:
import simpy
import random
import numpy as np
from simpy.events import Event

In [2]:
seed = 978
random.seed(seed)

# Arrival mean                              --EXPONENTIAL
interarrival_mean = 6 

# Operator 1 mean and std for service time  --LOGNORMAL
m = 12 
s = 6
M = np.log(m**2/np.sqrt(m**2+s**2))
S = np.log((m**2+s**2)/m**2)

# Operator 2 service time range             --UNIFORM
service_range = [1,7]

# Voice recognition mean -                  --EXPONENTIAL
router_mean = 5

SHIFT = 480

ANSWERED_CALLS = 1000
FINISHED_T = 0
all_customers = []
operation_t = [0,0]
shift_number = 0
is_done = 0

In [3]:
def service(env, opr):
    if opr==operator1:
        time = random.lognormvariate(M,S)
        operation_t[0] += time
        yield env.timeout(time)
        
    elif opr==operator2:
        time = random.uniform(*service_range)
        operation_t[1] += time
        yield env.timeout(time)

In [4]:
class Customer(object):
    def __init__(self, name, env):
        self.env = env
        self.name = name
        self.arrival_t = self.env.now
        self.action = env.process(self.call())
        self.waiting_t = 0
        
    
    def call(self):
        global is_done
        print('%s initiated a call at %g' % (self.name, self.env.now))
        
        
        # Voice recognition and routing
        if router.is_idle():
            router.working_t += [self.env.now]    
        router.count += [self.name]
        
        yield self.env.timeout(random.expovariate(1/router_mean))
        if random.uniform(0,1) < .3:
            self.operator = operator1
        else:
            self.operator = operator2
            
        router.count.remove(self.name)
        if router.is_idle():
            router.idle_t += [self.env.now]
        
            
        # Voice recognition failure
        if random.uniform(0,1) < .1:
            print('%s hangs up the call at %g' % (self.name, self.env.now))
            #all_customers.append(self)
            self.operator = None
            is_done += 1
            if is_done == ANSWERED_CALLS:
                finished.succeed()
            return 
        
        print('%s is rooted to operator %d at %g' % (self.name,(self.operator==operator2)+1, self.env.now))
        
        with self.operator.request(0) as req:
            self.waiting_t = self.env.now
            result = yield req | env.timeout(10)
            # Reneging for 10 mins
            if req not in result:
                print('%s is left the operator %d after 10 minutes at %g' % (self.name,
                                                               (self.operator==operator2)+1,
                                                               self.env.now))
                #all_customers.append(self)
                self.operator = None
                is_done += 1
                if is_done == ANSWERED_CALLS:
                    finished.succeed()
                return 
            self.waiting_t = self.env.now - self.waiting_t
            print('%s is assigned to the operator %d at %g' % (self.name,
                                                               (self.operator==operator2)+1,
                                                               self.env.now))
            yield self.env.process(service(env,self.operator))
            print('%s is done with operator %d at %g' % (self.name,
                                                        (self.operator==operator2)+1,
                                                        self.env.now))
            #all_customers.append(self)
            is_done += 1
            if is_done == ANSWERED_CALLS:
                finished.succeed()

In [5]:
class robocall:
    def __init__(self,env):
        self.count = []
        self.working_t = []
        self.idle_t = [0]
        self.env = env
        
    def answer(self):
        return len(self.count) < 100
    
    def is_idle(self):
        return len(self.count) == 0
            
    def utilization(self):
        return sum([ w-i for i,w in zip(router.idle_t, router.working_t)]) / FINISHED_T

In [6]:
def breaks():
    return [random.uniform(0,SHIFT) for i in  range(np.random.poisson(8))]

In [7]:
def shift(env):
    global shift_number
    shift_number += 1
    
    [env.process(take_a_break(env, b, operator1)) for b in breaks()]
    [env.process(take_a_break(env, b, operator2)) for b in breaks()]

    yield env.timeout(SHIFT)
    print("the time %g" % env.now)

    env.process(shift(env))

In [8]:
def take_a_break(env, number, opr):
    yield env.timeout(number)
    with opr.request(1) as brk:
        yield brk
        
        if SHIFT*shift_number - env.now > 0:
            if SHIFT*shift_number - env.now >= 3:
                print("break started for operator %d at %g" % ((opr == operator2)+1 , env.now))
                yield env.timeout(3)
            else:
                print("break started for operator %d at %g" % ((opr == operator2)+1 , env.now))
                yield env.timeout(480*shift_number - env.now)
            print("break ended for operator %d at %g" % ((opr == operator2)+1 , env.now))

In [9]:
def customer_generator(env):
    start = True
    num_answered = 0;
    
    while(len(all_customers)<ANSWERED_CALLS):
        
        #Every 480 a new shift starts.
        if start:
            env.process(shift(env))
            start = False
            
        yield env.timeout(random.expovariate(1/interarrival_mean))
        
        # Voice recognition system limit
        if router.answer():
            num_answered += 1
            customer = Customer('Customer %s' %(num_answered), env)
            all_customers.append(customer)
            
    FINISHED_T = env.now 

In [10]:
env = simpy.Environment()

finished = Event(env)
router = robocall(env)

operator1 = simpy.PriorityResource(env, capacity = 1)
operator2 = simpy.PriorityResource(env, capacity = 1)

# env.process(customer_generator(env))
# env.run(finished) 
# FINISHED_T = env.now
# print(FINISHED_T)
# print(len(all_customers))

In [11]:
def simulation(CALLS=1000, seed=978, verbose=0):
    
    global FINISHED_T 
    global all_customers
    global operation_t
    global shift_number
    global ANSWERED_CALLS
    global is_done
    
    is_done = 0
    ANSWERED_CALLS = CALLS
    FINISHED_T = 0
    all_customers = []
    operation_t = [0,0]
    shift_number = 0
    random.seed(seed)
    
    global  env 
    global finished 
    global router 
    global operator1 
    global operator2 
    env = simpy.Environment()
    finished = Event(env)
    router = robocall(env)
    operator1 = simpy.PriorityResource(env, capacity = 1)
    operator2 = simpy.PriorityResource(env, capacity = 1)
    env.process(customer_generator(env))
    env.run(finished) 
    FINISHED_T = env.now
    
    
    customer_waiting = [c.waiting_t for c in all_customers if c.operator != None]
    if verbose:
        print('Voice Recognition System Utilization: ', router.utilization())
        print('Utilization Operator 1:', operation_t[0]/FINISHED_T, 
              '\nUtilization Operator 2:' ,operation_t[1]/FINISHED_T)
        print('Average Waiting Time: ', sum(customer_waiting)/len(customer_waiting))
        
    return (router.utilization(),  
            operation_t[0]/FINISHED_T, 
            operation_t[1]/FINISHED_T, 
            sum(customer_waiting)/len(customer_waiting))

In [12]:
%%time
seeds = [305, 451, 973, 867, 135, 231, 53, 199, 155, 747]
calls = [1000, 5000]
statistics_1000 = [simulation(1000,s) for s in seeds]
# statistics_5000 = [simulation(5000,s) for s in seeds]

Customer 1 initiated a call at 1.20969
Customer 2 initiated a call at 8.09865
Customer 2 is rooted to operator 2 at 9.60545
Customer 2 is assigned to the operator 2 at 9.60545
Customer 2 is done with operator 2 at 11.4914
Customer 3 initiated a call at 12.1788
Customer 4 initiated a call at 13.0203
Customer 4 is rooted to operator 2 at 15.1996
Customer 4 is assigned to the operator 2 at 15.1996
Customer 3 is rooted to operator 2 at 16.048
Customer 4 is done with operator 2 at 17.6968
Customer 3 is assigned to the operator 2 at 17.6968
Customer 1 hangs up the call at 19.5033
Customer 3 is done with operator 2 at 22.6211
Customer 5 initiated a call at 27.4135
Customer 5 is rooted to operator 2 at 28.3125
Customer 5 is assigned to the operator 2 at 28.3125
Customer 5 is done with operator 2 at 29.9632
Customer 6 initiated a call at 34.5842
Customer 6 is rooted to operator 1 at 38.8003
Customer 6 is assigned to the operator 1 at 38.8003
Customer 7 initiated a call at 40.7188
Customer 8 ini

Customer 393 is rooted to operator 1 at 2206.08
Customer 393 is assigned to the operator 1 at 2206.08
Customer 394 initiated a call at 2206.22
Customer 394 is rooted to operator 1 at 2206.35
Customer 392 is rooted to operator 2 at 2208.14
Customer 392 is assigned to the operator 2 at 2208.14
Customer 395 initiated a call at 2211.41
Customer 392 is done with operator 2 at 2212.85
Customer 393 is done with operator 1 at 2213.44
Customer 394 is assigned to the operator 1 at 2213.44
Customer 391 is rooted to operator 2 at 2213.54
Customer 391 is assigned to the operator 2 at 2213.54
Customer 396 initiated a call at 2213.56
Customer 397 initiated a call at 2215.52
Customer 391 is done with operator 2 at 2215.93
Customer 395 is rooted to operator 2 at 2216.97
Customer 395 is assigned to the operator 2 at 2216.97
Customer 397 is rooted to operator 2 at 2221.11
Customer 398 initiated a call at 2221.48
Customer 395 is done with operator 2 at 2222.09
Customer 397 is assigned to the operator 2 at

Customer 803 is done with operator 1 at 4506.44
Customer 809 is rooted to operator 2 at 4506.77
Customer 809 is assigned to the operator 2 at 4506.77
Customer 808 is rooted to operator 1 at 4511.76
Customer 808 is assigned to the operator 1 at 4511.76
Customer 809 is done with operator 2 at 4513.29
Customer 810 initiated a call at 4514.16
Customer 806 hangs up the call at 4514.69
Customer 811 initiated a call at 4522.33
Customer 810 is rooted to operator 2 at 4523.05
Customer 810 is assigned to the operator 2 at 4523.05
Customer 808 is done with operator 1 at 4524.55
Customer 810 is done with operator 2 at 4526.77
Customer 812 initiated a call at 4528.42
Customer 811 is rooted to operator 2 at 4529.54
Customer 811 is assigned to the operator 2 at 4529.54
Customer 813 initiated a call at 4531.54
Customer 812 is rooted to operator 2 at 4531.92
Customer 813 is rooted to operator 1 at 4531.96
Customer 813 is assigned to the operator 1 at 4531.96
Customer 811 is done with operator 2 at 4535

Customer 198 is done with operator 2 at 1032.77
Customer 201 initiated a call at 1033.02
Customer 202 initiated a call at 1033.74
Customer 202 is rooted to operator 2 at 1034.54
Customer 202 is assigned to the operator 2 at 1034.54
Customer 203 initiated a call at 1037.01
Customer 200 is rooted to operator 2 at 1037.11
Customer 201 is rooted to operator 2 at 1038.33
Customer 202 is done with operator 2 at 1039.75
Customer 200 is assigned to the operator 2 at 1039.75
Customer 204 initiated a call at 1040.65
Customer 203 is rooted to operator 1 at 1046.03
Customer 203 is assigned to the operator 1 at 1046.03
Customer 205 initiated a call at 1046.16
Customer 200 is done with operator 2 at 1046.63
Customer 201 is assigned to the operator 2 at 1046.63
Customer 204 is rooted to operator 2 at 1046.91
Customer 206 initiated a call at 1048.38
Customer 201 is done with operator 2 at 1053.59
Customer 204 is assigned to the operator 2 at 1053.59
Customer 207 initiated a call at 1055.69
Customer 20

Customer 553 initiated a call at 3441.97
Customer 553 is rooted to operator 1 at 3443.72
Customer 553 is assigned to the operator 1 at 3443.72
Customer 554 initiated a call at 3446.47
Customer 554 is rooted to operator 2 at 3446.98
Customer 554 is assigned to the operator 2 at 3446.98
Customer 554 is done with operator 2 at 3448.02
Customer 555 initiated a call at 3450.29
Customer 556 initiated a call at 3454.81
Customer 556 is rooted to operator 2 at 3457.13
Customer 556 is assigned to the operator 2 at 3457.13
Customer 556 is done with operator 2 at 3458.5
Customer 553 is done with operator 1 at 3459.21
Customer 555 is rooted to operator 2 at 3460.31
Customer 555 is assigned to the operator 2 at 3460.31
Customer 555 is done with operator 2 at 3462.99
Customer 557 initiated a call at 3473.78
Customer 557 is rooted to operator 2 at 3474.71
Customer 557 is assigned to the operator 2 at 3474.71
Customer 557 is done with operator 2 at 3475.86
Customer 558 initiated a call at 3477.34
Custo

...

break ended for operator 1 at 3471.38
Customer 587 is rooted to operator 2 at 3474.71
Customer 586 is done with operator 2 at 3474.86
Customer 585 is assigned to the operator 2 at 3474.86
Customer 585 is done with operator 2 at 3476.22
Customer 587 is assigned to the operator 2 at 3476.22
Customer 588 initiated a call at 3477.27
Customer 587 is done with operator 2 at 3478.42
Customer 588 hangs up the call at 3481.49
Customer 589 initiated a call at 3490.58
Customer 589 is rooted to operator 2 at 3492.02
Customer 589 is assigned to the operator 2 at 3492.02
Customer 589 is done with operator 2 at 3494.14
Customer 590 initiated a call at 3502.09
Customer 591 initiated a call at 3506.61
Customer 591 is rooted to operator 2 at 3506.74
Customer 591 is assigned to the operator 2 at 3506.74
Customer 590 is rooted to operator 2 at 3508.11
Customer 591 is done with operator 2 at 3513.35
Customer 590 is assigned to the operator 2 at 3513.35
Customer 590 is done with operator 2 at 3520.05
Custom

Customer 802 initiated a call at 4848.38
Customer 802 is rooted to operator 2 at 4850.08
Customer 802 is assigned to the operator 2 at 4850.08
Customer 798 is done with operator 1 at 4850.89
Customer 801 is rooted to operator 2 at 4854.64
Customer 802 is done with operator 2 at 4856.42
Customer 801 is assigned to the operator 2 at 4856.42
Customer 803 initiated a call at 4860.42
Customer 803 is rooted to operator 1 at 4862.07
Customer 803 is assigned to the operator 1 at 4862.07
Customer 801 is done with operator 2 at 4862.85
break started for operator 2 at 4862.85
Customer 804 initiated a call at 4863.06
break ended for operator 2 at 4865.85
break started for operator 2 at 4865.85
Customer 805 initiated a call at 4867.33
break ended for operator 2 at 4868.85
break started for operator 2 at 4868.85
Customer 804 hangs up the call at 4870.91
Customer 806 initiated a call at 4871.19
break ended for operator 2 at 4871.85
Customer 803 is done with operator 1 at 4872.97
Customer 807 initiate

Customer 271 is rooted to operator 1 at 1636.76
Customer 271 is assigned to the operator 1 at 1636.76
Customer 272 initiated a call at 1637.65
Customer 270 is rooted to operator 1 at 1639.25
Customer 273 initiated a call at 1641.9
Customer 274 initiated a call at 1642.1
Customer 273 is rooted to operator 1 at 1642.41
Customer 271 is done with operator 1 at 1648.51
Customer 270 is assigned to the operator 1 at 1648.51
Customer 273 is left the operator 1 after 10 minutes at 1652.41
Customer 272 is rooted to operator 2 at 1653.27
Customer 272 is assigned to the operator 2 at 1653.27
Customer 272 is done with operator 2 at 1657.55
Customer 270 is done with operator 1 at 1659.92
Customer 274 is rooted to operator 1 at 1660.4
Customer 274 is assigned to the operator 1 at 1660.4
break started for operator 2 at 1668.72
break ended for operator 2 at 1671.72
Customer 274 is done with operator 1 at 1671.89
Customer 275 initiated a call at 1673.05
Customer 275 is rooted to operator 2 at 1679.73
Cu

Customer 638 initiated a call at 3819.86
break ended for operator 1 at 3820.07
break started for operator 1 at 3820.07
Customer 639 initiated a call at 3821.02
Customer 636 is done with operator 2 at 3821.43
Customer 639 is rooted to operator 2 at 3822.56
Customer 639 is assigned to the operator 2 at 3822.56
break ended for operator 1 at 3823.07
Customer 638 is rooted to operator 2 at 3824.77
Customer 639 is done with operator 2 at 3825.22
Customer 638 is assigned to the operator 2 at 3825.22
break started for operator 1 at 3826.5
Customer 640 initiated a call at 3828.72
break ended for operator 1 at 3829.5
Customer 641 initiated a call at 3830.16
Customer 641 is rooted to operator 2 at 3831.68
Customer 638 is done with operator 2 at 3831.96
Customer 641 is assigned to the operator 2 at 3831.96
Customer 640 is rooted to operator 1 at 3832.74
Customer 640 is assigned to the operator 1 at 3832.74
Customer 642 initiated a call at 3833.14
Customer 637 is rooted to operator 2 at 3833.44
Cus

Customer 888 is assigned to the operator 2 at 5266.22
Customer 889 is done with operator 1 at 5266.47
Customer 888 is done with operator 2 at 5268.17
Customer 890 initiated a call at 5274.94
break started for operator 2 at 5278.8
the time 5280
break ended for operator 2 at 5280
Customer 890 is rooted to operator 2 at 5280.14
Customer 890 is assigned to the operator 2 at 5280.14
Customer 890 is done with operator 2 at 5284.98
Customer 891 initiated a call at 5290.17
Customer 891 is rooted to operator 2 at 5291.38
Customer 891 is assigned to the operator 2 at 5291.38
Customer 892 initiated a call at 5295.4
Customer 891 is done with operator 2 at 5297.51
Customer 892 hangs up the call at 5297.89
Customer 893 initiated a call at 5304.51
Customer 893 is rooted to operator 2 at 5304.72
Customer 893 is assigned to the operator 2 at 5304.72
Customer 893 is done with operator 2 at 5311.25
Customer 894 initiated a call at 5317.52
Customer 895 initiated a call at 5320.07
Customer 896 initiated a 

In [13]:
for i in statistics_1000:
    for j in i:
        print(j, end= " ")
    print()

0.38418484748814646 0.44057402076170565 0.4436106416408834 1.8378061670243426 
0.4501279065268949 0.3763026779748624 0.42292595729342397 1.682768777973965 
0.4611119842189989 0.40210290622102035 0.3911629462020558 1.7701280951762535 
0.4537846334636767 0.43354442843697105 0.377960111969438 1.6001318482266196 
0.41383245636416605 0.4308876450081478 0.41220981497324133 1.572380008732588 
0.4491836444212835 0.4287533009037977 0.4044235738575419 1.600178762733095 
0.42627152531612694 0.43043048993795346 0.3995864817465923 1.51729886659867 
0.43194690825533816 0.39494439181392854 0.42590817363069094 1.6091302936138512 
0.42940078012699756 0.4549189955438388 0.42543739381049617 1.8956056635888971 
0.41406014261494756 0.424361683420326 0.4087968568433934 1.6553424955799916 
